<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/sample_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Saving a 1000 dev set for manual coding



In [0]:
DATASET_NAME = "status_posts"
MODEL_NAME = "status_posts"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 20
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=89d2c263bb5b69dc30d575634f8c2f1a1b9ca8bb0f98933bc88a11bd1f26b728
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# Get 1000 random indices for the dev set.
rand_index = np.random.randint(0,len(dev_df),1000)
rand_index=rand_index.tolist()

# Create a new dataframe for manual coding of 1000 random respones.
sample_1000 = dev_df.iloc[rand_index]
sample_1000 = sample_1000[['original_idx','response_text']]


In [0]:
# Download the new dataframe to local machine.
from google.colab import files

sample_1000.to_csv('sample_1000.csv',encoding='utf-8-sig', header=True, index=False)
files.download('sample_1000.csv') 


## Analysis of manually coded data

In [7]:
# Get manually coded csv files.
import io 

uploaded = files.upload()
sample_1000_LP = pd.read_csv(io.BytesIO(uploaded['sample_1000_LP.csv']))

Saving sample_1000.csv to sample_1000.csv


In [11]:
sample_1000_LP.LP_Gender.value_counts()

M    722
F    278
Name: LP_Gender, dtype: int64

In [0]:
orig_index_list = sample_1000_LP['original_idx'].to_list()

In [15]:
len(orig_index_list)

1000

In [0]:
dev_sample_1000 = dev_df[dev_df['original_idx'].isin(orig_index_list)]

In [14]:
# Not sure why one row is missing...
len(dev_sample_1000)

999

In [0]:
dev_sample_1000 = pd.merge(dev_sample_1000, sample_1000_LP[['original_idx','LP_Gender']], on='original_idx', how = 'left')

In [0]:
def F_to_W(gender):
  if gender == 'F':
    return 'W'
  else:
    return 'M'
  
dev_sample_1000['LP_Gender'] = dev_sample_1000['LP_Gender'].apply(F_to_W)


In [0]:
LP_correct = dev_sample_1000[dev_sample_1000['LP_Gender']==dev_sample_1000['op_gender']]
LP_incorrect = dev_sample_1000[dev_sample_1000['LP_Gender']!=dev_sample_1000['op_gender']]

In [30]:
len(LP_correct)

746

In [31]:
LP_correct.LP_Gender.value_counts()

M    664
W     82
Name: LP_Gender, dtype: int64

In [37]:
LP_correct.op_name.value_counts()


Marco Rubio               228
John McCain               105
Kirsten Gillibrand         42
Sherrod Brown              36
Tim Kaine                  35
Paul Ryan                  28
Ted Poe                    21
Ron Wyden                  20
Cathy McMorris Rodgers     20
Mark Takano                17
Kenny Marchant             16
Paul Gosar                 16
Jeff Flake                 15
Joe Barton                 14
Steve King                 12
Bill Huizenga              11
Peter DeFazio               8
Ryan Zinke                  8
Ann Wagner                  7
Keith Rothfus               7
Tim Walberg                 7
Daniel Webster              6
Blake Farenthold            6
Mike Crapo                  6
Michael McCaul              5
Betty McCollum              4
Richard Shelby              4
Steve Scalise               4
Don Beyer                   4
Sam Graves                  4
Ted Cruz                    4
Marc Veasey                 3
Pete Visclosky              3
Hal Rogers

In [32]:
LP_correct.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text_x,op_name,op_category,response_text_y,LP_Gender,ER_Gender
0,2119298,38794897,M,36268,Sara,thank you for reading king's letter today and standing by your colleague u.s. senator elizabeth ...,Sherrod Brown,Congress_Democratic,thank you for reading king's letter today and standing by your colleague u.s. senator elizabeth ...,M,NaN
1,2123236,38794897,M,36274,Marcy,why are you confirming carson?!?!,Sherrod Brown,Congress_Democratic,why are you confirming carson?!?!,M,NaN
2,2127010,38794897,M,36293,Mary,"those of us know better, know that is not true. god bless the ignorant et those who chose to be...",Sherrod Brown,Congress_Democratic,"those of us know better, know that is not true. god bless the ignorant et those who chose to be...",M,NaN
4,2129211,38794897,M,36314,Zot,do your job and protect consumers by voting against the roberts/stabanow dark act to deny consum...,Sherrod Brown,Congress_Democratic,do your job and protect consumers by voting against the roberts/stabanow dark act to deny consum...,M,NaN
6,2130808,38794897,M,36328,Valerie,he has going to have a rude awakening,Sherrod Brown,Congress_Democratic,he has going to have a rude awakening,M,NaN


In [34]:
LP_incorrect.head(10)

,original_idx,op_id,op_gender,post_id,responder_id,response_text_x,op_name,op_category,response_text_y,LP_Gender,ER_Gender
3,2128047,38794897,M,36304,Rebecca,"clinton wants to privatize social security, too. are you saying you will not support her effort...",Sherrod Brown,Congress_Democratic,"clinton wants to privatize social security, too. are you saying you will not support her effort...",W,NaN
5,2129555,38794897,M,36318,Debbie,"senator, why have you sold your soul to corp. and wall street by endorsing hillary clinton. you...",Sherrod Brown,Congress_Democratic,"senator, why have you sold your soul to corp. and wall street by endorsing hillary clinton. you...",W,NaN
7,2130916,38794897,M,36329,Kathryn,do not think she will get my vote.,Sherrod Brown,Congress_Democratic,do not think she will get my vote.,W,NaN
10,2135200,38794897,M,36365,Eric,my faith tells me that we all need and deserve second chances. i urge you to support reauthoriza...,Sherrod Brown,Congress_Democratic,my faith tells me that we all need and deserve second chances. i urge you to support reauthoriza...,W,NaN
13,2137454,38794897,M,36407,Linda,"charlotte chipps ... you sound as if you have a ""chipp"" on your shoulder, maybe! but happy than...",Sherrod Brown,Congress_Democratic,"charlotte chipps ... you sound as if you have a ""chipp"" on your shoulder, maybe! but happy than...",W,NaN
15,2141931,38794897,M,36481,Mimi,"i love connie, too.",Sherrod Brown,Congress_Democratic,"i love connie, too.",W,NaN
27,2156094,38794897,M,36747,Norma,this is great news but if the republicans keep trying to destroy our country will it really matter?,Sherrod Brown,Congress_Democratic,this is great news but if the republicans keep trying to destroy our country will it really matter?,W,NaN
32,2161739,38794897,M,36826,Gwen,our privilege to vote for you and our good fortune to have you working for us.,Sherrod Brown,Congress_Democratic,our privilege to vote for you and our good fortune to have you working for us.,W,NaN
34,2172894,38794897,M,36883,Renee,i wish i could vote for you but i do support your work and your ideals!,Sherrod Brown,Congress_Democratic,i wish i could vote for you but i do support your work and your ideals!,W,NaN
39,2192417,38794897,M,37895,Jeffrey,wow in one month the unemployment went down to 9% way to go gop,Sherrod Brown,Congress_Democratic,wow in one month the unemployment went down to 9% way to go gop,W,NaN


In [33]:
# Number of M/F incorrect guesses.
LP_incorrect.LP_Gender.value_counts()

W    196
M     58
Name: LP_Gender, dtype: int64

In [38]:
LP_incorrect.op_name.value_counts()

Marco Rubio               36
Kirsten Gillibrand        27
John McCain               25
Mark Takano               14
Cathy McMorris Rodgers    14
Sherrod Brown             13
Paul Ryan                  9
Ted Poe                    9
Paul Gosar                 8
Don Beyer                  8
Ron Wyden                  7
Steve King                 7
Tim Kaine                  7
Ann Wagner                 6
Joe Barton                 5
Jeff Flake                 5
Ryan Zinke                 5
Robert Aderholt            3
Michael McCaul             3
Robin Kelly                3
Mike Crapo                 3
Kenny Marchant             3
Tim Walberg                3
Richard Shelby             3
Sam Graves                 2
Stephen F. Lynch           2
Anna Eshoo                 2
Bill Huizenga              2
Jaime Herrera Beutler      2
Steve Scalise              2
Peter DeFazio              2
John Garamendi             2
Blake Farenthold           2
Daniel Webster             2
Catherine Cort

In [0]:
# Lorrel incorrectly guessed male.
LP_false_M = LP_incorrect[LP_incorrect['LP_Gender']=='M']
# Lorrel incorrectly guessed female.
LP_false_F = LP_incorrect[LP_incorrect['LP_Gender']=='W']


In [36]:
LP_false_M.head(10)

,original_idx,op_id,op_gender,post_id,responder_id,response_text_x,op_name,op_category,response_text_y,LP_Gender,ER_Gender
50,2256755,47977187,W,44147,Chris,only democrats need to conceal their party affiliation,Catherine Cortez Masto,Congress_Democratic,only democrats need to conceal their party affiliation,M,NaN
169,2650001,13876473,W,69517,Anna,take that hand away ushhhhh,Carolyn Maloney,Congress_Democratic,take that hand away ushhhhh,M,NaN
173,3126674,23984618,W,82989,Tami,guess we sent them the cash for their next big attack on american soil.,Ann Wagner,Congress_Republican,guess we sent them the cash for their next big attack on american soil.,M,NaN
176,3135248,23984618,W,83407,Brandy,harry is scary!!,Ann Wagner,Congress_Republican,harry is scary!!,M,NaN
179,3138132,23984618,W,83535,Phyllis,"its not blind allegiance, its being smart enough to have a solid plan before execution...its get...",Ann Wagner,Congress_Republican,"its not blind allegiance, its being smart enough to have a solid plan before execution...its get...",M,NaN
180,3138980,23984618,W,83574,Robert,"dear mr. groth: you obviously have never met, do not know or cannot intellectually grasp who ann...",Ann Wagner,Congress_Republican,"dear mr. groth: you obviously have never met, do not know or cannot intellectually grasp who ann...",M,NaN
181,3141798,23984618,W,83646,Larry,"why do we not have anyone in congress or senate to start impeachment against obama, he is the bi...",Ann Wagner,Congress_Republican,"why do we not have anyone in congress or senate to start impeachment against obama, he is the bi...",M,NaN
184,3150003,23984618,W,83797,Stephanie,would have said pappy's up until we tried l'il mickey's in st. peters. pappy's has better sides...,Ann Wagner,Congress_Republican,would have said pappy's up until we tried l'il mickey's in st. peters. pappy's has better sides...,M,NaN
196,3361581,76269753,W,103744,Rob,"with the strictest gun laws in america, banning everything you want banned, explain chicago!, hi...",Dianne Feinstein,Congress_Democratic,"with the strictest gun laws in america, banning everything you want banned, explain chicago!, hi...",M,NaN
502,6087382,19189340,W,208352,Vicki,thats for us,Robin Kelly,Congress_Democratic,thats for us,M,NaN


In [39]:
LP_false_F.head(10)

,original_idx,op_id,op_gender,post_id,responder_id,response_text_x,op_name,op_category,response_text_y,LP_Gender,ER_Gender
3,2128047,38794897,M,36304,Rebecca,"clinton wants to privatize social security, too. are you saying you will not support her effort...",Sherrod Brown,Congress_Democratic,"clinton wants to privatize social security, too. are you saying you will not support her effort...",W,NaN
5,2129555,38794897,M,36318,Debbie,"senator, why have you sold your soul to corp. and wall street by endorsing hillary clinton. you...",Sherrod Brown,Congress_Democratic,"senator, why have you sold your soul to corp. and wall street by endorsing hillary clinton. you...",W,NaN
7,2130916,38794897,M,36329,Kathryn,do not think she will get my vote.,Sherrod Brown,Congress_Democratic,do not think she will get my vote.,W,NaN
10,2135200,38794897,M,36365,Eric,my faith tells me that we all need and deserve second chances. i urge you to support reauthoriza...,Sherrod Brown,Congress_Democratic,my faith tells me that we all need and deserve second chances. i urge you to support reauthoriza...,W,NaN
13,2137454,38794897,M,36407,Linda,"charlotte chipps ... you sound as if you have a ""chipp"" on your shoulder, maybe! but happy than...",Sherrod Brown,Congress_Democratic,"charlotte chipps ... you sound as if you have a ""chipp"" on your shoulder, maybe! but happy than...",W,NaN
15,2141931,38794897,M,36481,Mimi,"i love connie, too.",Sherrod Brown,Congress_Democratic,"i love connie, too.",W,NaN
27,2156094,38794897,M,36747,Norma,this is great news but if the republicans keep trying to destroy our country will it really matter?,Sherrod Brown,Congress_Democratic,this is great news but if the republicans keep trying to destroy our country will it really matter?,W,NaN
32,2161739,38794897,M,36826,Gwen,our privilege to vote for you and our good fortune to have you working for us.,Sherrod Brown,Congress_Democratic,our privilege to vote for you and our good fortune to have you working for us.,W,NaN
34,2172894,38794897,M,36883,Renee,i wish i could vote for you but i do support your work and your ideals!,Sherrod Brown,Congress_Democratic,i wish i could vote for you but i do support your work and your ideals!,W,NaN
39,2192417,38794897,M,37895,Jeffrey,wow in one month the unemployment went down to 9% way to go gop,Sherrod Brown,Congress_Democratic,wow in one month the unemployment went down to 9% way to go gop,W,NaN
